# Trying to optimize my code 
- The integral size sample computing the expected value is super slow, i need to make it faster 

In [1]:
import sys
import os

sys.path.append("/Users/filiprolenec/Desktop/MT/MTpython/src")

In [2]:
import operator
from typing import List

import numpy as np

from gas_example.enum_types import Action, PowerplantState
from gas_example.optimization.basis_function import uf_2, uf_2_inv
from gas_example.setup import BORROW_RATE_EPOCH, RISK_FREE_RATE_EPOCH
from gas_example.simulation.state import get_valid_actions, action_is_invalid, get_installed_mw, get_next_plant_state

from gas_example.setup import get_epoch_rate, GAS_VOL, CO2_VOL, POWER_VOL, \
    POWERPLANT_COST, MAINTENANCE_COST_PER_MW, HOURS_IN_EPOCH, BORROW_RATE_YEAR, RISK_FREE_RATE_YEAR, BORROW_RATE_EPOCH, \
    RISK_FREE_RATE_EPOCH, EPOCHS_IN_YEAR

import seaborn as sns
from os import listdir
from os.path import isfile, join
import gas_example.simulation.strategy as strategy
sns.set()

from progressbar import progressbar

In [7]:
class State:
    def __init__(self, gas_price: float, co2_price: float, power_price: float,
                 plant_state: PowerplantState, balance: float):
        self.gas_price = gas_price
        self.co2_price = co2_price
        self.power_price = power_price
        self.plant_state = plant_state
        self.balance = balance
        
        self.price_coefs_dict = get_price_coefs_dict(INTEGRAL_SAMPLE_SIZE)

    # Used to get faster expected utility, vector form giving us faster results
    def get_spark_prices_and_fcfs(self, action):
        gas_prices = self.gas_price*self.price_coefs_dict["Gas"]
        co2_prices = self.co2_price*self.price_coefs_dict["CO2"]
        power_prices = self.power_price*self.price_coefs_dict["Power"]
    
        
        plant_state = [get_next_plant_state(self, action)]*INTEGRAL_SAMPLE_SIZE

        spark_prices = power_prices-co2_prices-gas_prices
        
        fcfs = compute_fcfs(spark_prices,
                          self.plant_state,
                          action)

        return spark_prices, fcfs
    
    # this is for the actual simulation.
    def get_new_state_and_fcf(self, action):
        gas_price = get_next_price(self.gas_price, GAS_VOL)
        co2_price = get_next_price(self.co2_price, CO2_VOL)
        power_price = get_next_price(self.power_price, POWER_VOL)
        plant_state = get_next_plant_state(self, action)

        fcf = compute_fcf(self,
                          self.plant_state,
                          action)

        balance = round(update_balance(self.balance, fcf))

        return State(gas_price, co2_price, power_price, plant_state, balance), fcf
    
    def to_dict(self):
        return self.__dict__
    
    def get_spark_price(self):
        return self.power_price - self.co2_price - self.gas_price - MAINTENANCE_COST_PER_MW


In [8]:
def get_best_action(state: State, future_vf, print_details=False):
    valid_actions = get_valid_actions(state)

    exp_utility_per_action = {}
    for action in valid_actions:
        # We would like to compute expected value, we approximate by average of samples.
        utility_realizations = get_utility_realizations(state, action, future_vf)

        exp_utility_per_action[action] = np.mean(utility_realizations)
        
    if print_details:
        print(f"Spark: {state.get_spark_price()}")
        print(exp_utility_per_action)
        print("\n")

    best_action = max(exp_utility_per_action.items(), key=operator.itemgetter(1))[0]

    return best_action, exp_utility_per_action[best_action]

In [9]:
[f for f in listdir("../saved_vfs") if isfile(join("../saved_vfs", f))]


['vfs_12_21_2020.pkl',
 'vfs_12_20_2020.pkl',
 'vfs_2020-12-24_H10.pkl',
 'vfs_2020-12-28_H18.pkl',
 'vfs_2020-12-25_H09.pkl',
 'vfs_12_22_2020.pkl']

In [10]:
opt_strategy = strategy.OptimalStrategy("../saved_vfs/vfs_2020-12-28_H18.pkl")

In [11]:
CO2_VOL = 0.10
GAS_VOL = 0.12
POWER_VOL = 0.15
EPOCHS_IN_YEAR = 12

In [12]:
def get_price_coefs_dict(number_of_samples):
    asset_names = ["CO2", "Gas", "Power"]
    sigmas = [CO2_VOL, GAS_VOL, POWER_VOL]
    price_coefs_dict = {}

    for i, name in enumerate(asset_names):
        dt = 1/EPOCHS_IN_YEAR
        price_coefs_dict[name] = np.exp((0 - sigmas[i] ** 2 / 2) * dt + sigmas[i] * np.random.normal(0, np.sqrt(dt), number_of_samples))

    return price_coefs_dict


In [14]:
def compute_fcfs(spark_prices, 
                plant_state: PowerplantState,
                action: Action):
    
    single_profit = 0

    # Building new capacity costs money
    if action == Action.IDLE_AND_BUILD or action == Action.RUN_AND_BUILD:
        single_profit = - POWERPLANT_COST

    installed_mw = get_installed_mw(plant_state)


    single_profit -= installed_mw * MAINTENANCE_COST_PER_MW * HOURS_IN_EPOCH

    # Making profit if action is to run:
    profits = [single_profit]*INTEGRAL_SAMPLE_SIZE
    
    if action == Action.RUN_AND_BUILD or action == Action.RUN:
        profits += spark_prices * installed_mw * HOURS_IN_EPOCH

    return profits

In [15]:
def get_utility_realizations(state: State, action: Action, future_vf):
    spark_prices, fcfs = state.get_spark_prices_and_fcfs(action)
    new_powerplant_state = get_next_plant_state(state, action)
    
    future_vf_utilities = future_vf.compute_value(new_powerplant_state, spark_prices)
    
    future_vf_money_equivalents = INVERSE_UTILITY_FUNCTION_V(future_vf_utilities)

    updated_balances = [fcf+state.balance for fcf in fcfs]
    
    balance_future_vf_pairs = [[a, b] for a,b in zip(updated_balances, future_vf_money_equivalents)]

    pce_realizations = [pce - state.balance for pce in pce_v(balance_future_vf_pairs)]

    utility_realizations = np.round(UTILITY_FUNCTION_V(pce_realizations), 2)
    
    return utility_realizations

In [16]:
UTILITY_FUNCTION_V = np.vectorize(uf_2)
INVERSE_UTILITY_FUNCTION_V = np.vectorize(uf_2_inv)


In [17]:
def pce(fcfs):
    balance = 0

    r_b = BORROW_RATE_EPOCH
    r_r = RISK_FREE_RATE_EPOCH

    for fcf in fcfs:
        balance += fcf
        if balance < 0:
            balance = balance * r_b
        else:
            balance = balance * r_r

    if balance < 0:
        return balance / r_b ** (len(fcfs))
    else:
        return balance / r_r ** (len(fcfs))

In [18]:
def pce_v(fcfs_v): 
    pces = []
    for fcfs in fcfs_v: 
        pces.append(pce(fcfs))
    return pces

In [20]:
vfs = opt_strategy.vfs

In [27]:
INTEGRAL_SAMPLE_SIZE = 100

In [28]:
init_state=State(10, 25, 400, PowerplantState.STAGE_1, 0)


In [30]:
get_utility_realizations(init_state,Action.RUN, vfs[20])

array([27157.62, 25141.32, 25047.55, 25815.17, 26551.8 , 23696.85,
       24837.86, 25496.76, 25355.55, 23806.88, 26698.68, 27302.94,
       25448.41, 25958.22, 25597.58, 25394.57, 23682.26, 25164.22,
       25499.51, 25407.7 , 26359.03, 24529.34, 27447.33, 27428.92,
       27716.59, 26611.82, 25647.22, 26364.36, 25561.76, 27083.17,
       26366.97, 26815.97, 27370.2 , 25777.9 , 24952.44, 26355.22,
       27145.48, 24966.01, 25834.7 , 25784.41, 27628.14, 27653.75,
       24305.15, 24334.52, 26479.9 , 25528.55, 24687.38, 26919.78,
       26079.81, 25499.61, 25275.63, 27080.74, 26323.81, 27367.05,
       24223.48, 24757.42, 25922.23, 27904.09, 27407.59, 25528.07,
       26669.58, 23785.68, 23103.17, 24984.67, 27045.44, 25893.35,
       25154.99, 25163.95, 27007.04, 23424.97, 25328.34, 26135.46,
       26216.81, 24975.13, 25677.45, 25421.03, 27178.95, 30460.01,
       24641.73, 25357.16, 26981.88, 25814.47, 25974.84, 25814.98,
       25789.77, 23771.66, 27447.65, 24414.54, 26361.92, 27165

In [31]:
vfs = opt_strategy.vfs

In [43]:
import pendulum
import pandas as pd

In [44]:
pendulum.now()

DateTime(2020, 12, 29, 12, 34, 13, 423167, tzinfo=Timezone('Europe/Prague'))

In [45]:
for i in range(10): 
    print(i)
    before_time = pd.Timestamp.now()
    INTEGRAL_SAMPLE_SIZE = 10**i
    init_state=State(10, 25, 400, PowerplantState.STAGE_1, 0)

    get_best_action(init_state, opt_strategy.vfs[1], True)
    print((pd.Timestamp.now()-before_time))

0
Spark: 359
{<Action.DO_NOTHING: 0>: -634.43, <Action.RUN: 1>: 26258.8, <Action.RUN_AND_BUILD: 2>: 21644.49, <Action.IDLE_AND_BUILD: 3>: -6986.45}


0 days 00:00:00.001377
1
Spark: 359
{<Action.DO_NOTHING: 0>: -634.4300000000001, <Action.RUN: 1>: 25506.932, <Action.RUN_AND_BUILD: 2>: 20881.988999999998, <Action.IDLE_AND_BUILD: 3>: -6986.45}


0 days 00:00:00.001557
2
Spark: 359
{<Action.DO_NOTHING: 0>: -634.4300000000001, <Action.RUN: 1>: 25748.883200000004, <Action.RUN_AND_BUILD: 2>: 21127.1427, <Action.IDLE_AND_BUILD: 3>: -6986.449999999996}


0 days 00:00:00.004934
3
Spark: 359
{<Action.DO_NOTHING: 0>: -634.43, <Action.RUN: 1>: 25740.445389999997, <Action.RUN_AND_BUILD: 2>: 21118.53327, <Action.IDLE_AND_BUILD: 3>: -6986.449999999998}


0 days 00:00:00.022761
4
Spark: 359
{<Action.DO_NOTHING: 0>: -634.43, <Action.RUN: 1>: 25755.977190999998, <Action.RUN_AND_BUILD: 2>: 21134.298159999995, <Action.IDLE_AND_BUILD: 3>: -6986.449999999999}


0 days 00:00:00.191589
5
Spark: 359
{<Action.D

KeyboardInterrupt: 

In [48]:
0.19*200*300/60/60

3.1666666666666665

In [ ]:
INTEGRAL_SAMPLE_SIZE = 10000
get_best_action(init_state, opt_strategy.vfs[1], True)